Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = '../data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
# Only run this cell once
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  print('np.arange(num_labels): ', np.arange(num_labels))
  print('labels[:,None]: ', labels[:, None])
  labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
  print('labels: ', labels)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

np.arange(num_labels):  [0 1 2 3 4 5 6 7 8 9]
labels[:,None]:  [[4]
 [9]
 [6]
 ..., 
 [2]
 [4]
 [4]]
labels:  [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
np.arange(num_labels):  [0 1 2 3 4 5 6 7 8 9]
labels[:,None]:  [[1]
 [9]
 [3]
 ..., 
 [8]
 [9]
 [8]]
labels:  [[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
np.arange(num_labels):  [0 1 2 3 4 5 6 7 8 9]
labels[:,None]:  [[3]
 [6]
 [4]
 ..., 
 [6]
 [9]
 [0]]
labels:  [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 1.  0.  0. ...,  0.  0.  0.]]
Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (1

In [7]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Problem 1 - Logistic with L2 regularization

In [21]:
# logistic

batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + 0.01 * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [22]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 48.413712
Minibatch accuracy: 7.0%
Validation accuracy: 8.4%
Minibatch loss at step 500: 0.723762
Minibatch accuracy: 85.9%
Validation accuracy: 81.5%
Minibatch loss at step 1000: 0.801756
Minibatch accuracy: 80.5%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 0.566287
Minibatch accuracy: 83.6%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 0.648371
Minibatch accuracy: 88.3%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 0.782499
Minibatch accuracy: 78.9%
Validation accuracy: 81.1%
Minibatch loss at step 3000: 0.784398
Minibatch accuracy: 80.5%
Validation accuracy: 81.4%
Test accuracy: 88.6%


### Problem 1 - Neural Network with L2 regularization

In [25]:
# Neural network

batch_size = 128
hidden_nodes_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes_size]))
    
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_size, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  tmp_logits = tf.matmul(tf_train_dataset, weights_1) + biases_1
  hidden_layer_relu_logits = tf.nn.relu(tmp_logits)
    
  logits = tf.matmul(hidden_layer_relu_logits, weights_2) + biases_2
    
  # Does the scaling hyperparameter for l2 loss work better if it's twice as small here?
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
    + 0.01 * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), 
      weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), 
      weights_2) + biases_2)

In [26]:
# This is a copy paste of the training cell above:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3454.936035
Minibatch accuracy: 8.6%
Validation accuracy: 21.1%
Minibatch loss at step 500: 21.194702
Minibatch accuracy: 87.5%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 0.947421
Minibatch accuracy: 80.5%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 0.584872
Minibatch accuracy: 87.5%
Validation accuracy: 83.6%
Minibatch loss at step 2000: 0.612600
Minibatch accuracy: 89.1%
Validation accuracy: 83.3%
Minibatch loss at step 2500: 0.718377
Minibatch accuracy: 84.4%
Validation accuracy: 83.2%
Minibatch loss at step 3000: 0.765629
Minibatch accuracy: 83.6%
Validation accuracy: 83.5%
Test accuracy: 90.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [36]:
batch_size = 128
hidden_nodes_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes_size]))
    
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_size, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  tmp_logits = tf.matmul(tf_train_dataset, weights_1) + biases_1
  hidden_layer_relu_logits = tf.nn.relu(tmp_logits)
    
  logits = tf.matmul(hidden_layer_relu_logits, weights_2) + biases_2
    
  # Does the scaling hyperparameter for l2 loss work better if it's twice as small here?
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
    + 0.01 * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), 
      weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), 
      weights_2) + biases_2)

In [32]:
print(train_dataset.shape)
print(train_labels.shape)

(200000, 784)
(200000, 10)


In [39]:
# This is a copy paste of the training cell above:
num_steps = 1001
small_train_dataset = train_dataset[0:500,:]
small_train_labels = train_labels[0:500,:]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = small_train_dataset[offset:(offset + batch_size), :]
    batch_labels = small_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3483.867432
Minibatch accuracy: 9.4%
Validation accuracy: 36.2%
Minibatch loss at step 100: 1151.358276
Minibatch accuracy: 100.0%
Validation accuracy: 76.6%
Minibatch loss at step 200: 422.500977
Minibatch accuracy: 100.0%
Validation accuracy: 76.3%
Minibatch loss at step 300: 155.041733
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Minibatch loss at step 400: 56.914989
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 500: 21.019978
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 600: 7.902267
Minibatch accuracy: 100.0%
Validation accuracy: 79.1%
Minibatch loss at step 700: 3.118054
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Minibatch loss at step 800: 1.362667
Minibatch accuracy: 99.2%
Validation accuracy: 79.5%
Minibatch loss at step 900: 0.687645
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Minibatch loss at step 1000: 0.491553
Minibatch accuracy: 99.2%
Validat

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [9]:
# This has both l2 regularization and dropout.

batch_size = 128
hidden_nodes_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  keep_prob = tf.placeholder(tf.float32)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes_size]))
    
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_size, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  tmp_logits = tf.matmul(tf_train_dataset, weights_1) + biases_1
  hidden_layer_relu_logits = tf.nn.relu(tmp_logits)
  hidden_layer_relu_logits = tf.nn.dropout(hidden_layer_relu_logits, keep_prob)
    
  logits = tf.matmul(hidden_layer_relu_logits, weights_2) + biases_2
    
  # Does the scaling hyperparameter for l2 loss work better if it's twice as small here?
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), 
      weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), 
      weights_2) + biases_2)

In [10]:
# New param keep_prob in the dictionary.
num_steps = 1001
small_train_dataset = train_dataset[0:2000,:]
small_train_labels = train_labels[0:2000,:]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = small_train_dataset[offset:(offset + batch_size), :]
    batch_labels = small_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 461.951965
Minibatch accuracy: 10.9%
Validation accuracy: 31.3%
Minibatch loss at step 100: 3.729500
Minibatch accuracy: 93.0%
Validation accuracy: 79.4%
Minibatch loss at step 200: 10.796918
Minibatch accuracy: 94.5%
Validation accuracy: 80.2%
Minibatch loss at step 300: 2.666889
Minibatch accuracy: 96.9%
Validation accuracy: 80.2%
Minibatch loss at step 400: 0.324613
Minibatch accuracy: 98.4%
Validation accuracy: 79.8%
Minibatch loss at step 500: 1.089019
Minibatch accuracy: 98.4%
Validation accuracy: 79.1%
Minibatch loss at step 600: 9.192682
Minibatch accuracy: 94.5%
Validation accuracy: 80.0%
Minibatch loss at step 700: 1.445745
Minibatch accuracy: 98.4%
Validation accuracy: 81.0%
Minibatch loss at step 800: 0.000540
Minibatch accuracy: 100.0%
Validation accuracy: 80.3%
Minibatch loss at step 900: 0.279108
Minibatch accuracy: 99.2%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 0.191432
Minibatch accuracy: 99.2%
Validation accuracy: 8

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
